In [ ]:
!pip install tqdm pandas

In [8]:
import time
import requests
import pandas as pd
from tqdm import tqdm

CSV_FILE_PATH = "annotations.csv"  # Enter the path to your CSV file

df = pd.read_csv(CSV_FILE_PATH)

# Ensure the 'id' column exists
if "id" not in df.columns:
    raise ValueError("The CSV file must contain a column named 'id'.")

# Define the Bugzilla API URL
BUGZILLA_URL = "https://bugzilla.redhat.com"

# Function to get bug details from Bugzilla
def get_bug_details(bug_id):
    url = f"{BUGZILLA_URL}/rest/bug/{bug_id}"
    params = {"include_fields": "id,creator,creation_time,last_change_time,status,resolution,priority,severity,classification,component,summary"}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        bug_data = response.json().get("bugs", [{}])
        return bug_data[0] if bug_data and bug_data[0] else None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Fetch bug details with a progress bar
bug_details_list = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Fetching Bug Details"):
    bug_id = row["id"]
    bug_info = get_bug_details(bug_id)
    
    bug_details_list.append([
        bug_id,
        bug_info.get("creator", "") if bug_info else "Not Found",
        bug_info.get("creation_time", "") if bug_info else "",
        bug_info.get("last_change_time", "") if bug_info else "",
        f"{bug_info.get('status', '')} {bug_info.get('resolution', '')}".strip() if bug_info else "",
        bug_info.get("priority", "") if bug_info else "",
        bug_info.get("severity", "") if bug_info else "",
        bug_info.get("classification", "") if bug_info else "",
        bug_info.get("component", "") if bug_info else "",
        bug_info.get("summary", "") if bug_info else "",
    ])
    time.sleep(1)

# Create DataFrame and save to CSV
bug_df = pd.DataFrame(bug_details_list, columns=[
    "id", "reporter", "reporter-time", "last-modified", "status", "priority", "severity", "product", "component", "summary"
])
bug_df.to_csv(CSV_FILE_PATH, index=False)

print("Bug information successfully inserted into the CSV file.")


Fetching Bug Details: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.21s/it]

Bug information successfully inserted into the local CSV file.


In [7]:
import time
import requests
import pandas as pd
from tqdm import tqdm

CSV_FILE_PATH = "descriptions.csv"  # Enter the path to your CSV file

df = pd.read_csv(CSV_FILE_PATH)

# Ensure the 'id' column exists
if "id" not in df.columns:
    raise ValueError("The CSV file must contain a column named 'id'.")

# Define the Bugzilla API URL
BUGZILLA_URL = "https://bugzilla.redhat.com"

# Function to get bug details from Bugzilla
def get_bug_description(bug_id):
    url = f"{BUGZILLA_URL}/rest/bug/{bug_id}"
    params = {"include_fields": "id, description"}
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        bug_data = response.json().get("bugs", [{}])
        return bug_data[0] if bug_data and bug_data[0] else None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Fetch bug details with a progress bar
bug_details_list = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Fetching Bug Descriptions"):
    bug_id = row["id"]
    bug_info = get_bug_description(bug_id)
    
    bug_details_list.append([
        bug_id,
        bug_info.get("description", "") if bug_info else "",
    ])
    time.sleep(1)  # Delay to avoid rate limits

# Create DataFrame and save to CSV
bug_df = pd.DataFrame(bug_details_list, columns=[
    "id", "description"
])
bug_df.to_csv(CSV_FILE_PATH, index=False)

print("Bug details successfully inserted into the CSV file.")


Fetching Bug Descriptions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.21s/it]

Bug details successfully inserted into the CSV file.
